# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
Cities_DF = pd.read_csv("..\VacationPy\output_data\cities.csv")
Cities_DF.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapaa,22.0752,-159.3190,26.99,77,40,6.17,US,1651608887
1,1,barrow,71.2906,-156.7887,-5.99,86,100,5.14,US,1651608707
2,2,busselton,-33.6500,115.3333,13.33,68,84,4.85,AU,1651608889
3,3,mar del plata,-38.0023,-57.5575,16.08,59,0,6.17,AR,1651608890
4,4,vaini,-21.2000,-175.2000,23.09,88,20,2.57,TO,1651608890


In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#get locations and humidity
city_locations = Cities_DF[['Lat','Lng']]
humidity = Cities_DF['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                               point_radius = 1)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

In [5]:
Cities_DF

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapaa,22.0752,-159.3190,26.99,77,40,6.17,US,1651608887
1,1,barrow,71.2906,-156.7887,-5.99,86,100,5.14,US,1651608707
2,2,busselton,-33.6500,115.3333,13.33,68,84,4.85,AU,1651608889
3,3,mar del plata,-38.0023,-57.5575,16.08,59,0,6.17,AR,1651608890
4,4,vaini,-21.2000,-175.2000,23.09,88,20,2.57,TO,1651608890
...,...,...,...,...,...,...,...,...,...,...
546,546,poshekhonye,58.4993,39.1353,2.33,90,100,3.29,RU,1651609201
547,547,santa rosalia,27.3167,-112.2833,26.61,33,98,2.33,MX,1651609201
548,548,boqueirao,-7.5000,-36.1333,26.75,73,25,5.61,BR,1651609202
549,549,panyutyne,48.9333,36.2833,10.82,56,29,2.27,UA,1651609203


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrowing the list of cities with temperature in the range (20,30) and Maximum windspeed is 5 and cloudiness is 0.
hotel_df = Cities_DF.loc[(Cities_DF['Max Temp'] > 20) & (Cities_DF['Max Temp'] <30) & (Cities_DF['Wind Speed'] <=5)& (Cities_DF['Cloudiness'] == 0), :].dropna()
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,prado,-17.3411,-39.2208,25.18,73,0,3.11,BR,1651608908
44,44,nizwa,22.9333,57.5333,27.18,18,0,2.45,OM,1651608917
68,68,sao filipe,14.8961,-24.4956,22.95,76,0,4.32,CV,1651608933
107,107,cabo san lucas,22.8909,-109.9124,28.67,44,0,4.63,MX,1651608895
126,126,kingman,35.1894,-114.0530,23.96,8,0,4.12,US,1651608887


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get variables type from df
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params["location"] = f'{lat},{lng}'
        
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"In {city} closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"Missing field/result for {city}... skipping.")
        
    print("------------")

In prado closest hotel is POUSADA PONTA DE AREIA - PRADO - BAHIA - HOSPEDAGEM E HOTEL.
------------
In nizwa closest hotel is The View.
------------
In sao filipe closest hotel is Hotel Xaguate.
------------
In cabo san lucas closest hotel is Solmar Resort.
------------
In kingman closest hotel is Don Laughlin's Riverside Resort Hotel and Casino.
------------
In caravelas closest hotel is POUSADA PONTA DE AREIA - PRADO - BAHIA - HOSPEDAGEM E HOTEL.
------------
In zhuhai closest hotel is Hotel COZi ‧ Resort.
------------
In sur closest hotel is Ras Al Jinz Turtle Reserve.
------------
In catamarca closest hotel is La Ceiba.
------------
In veraval closest hotel is Hotel Riverfront.
------------
In santiago del estero closest hotel is Hilton Garden Inn Santiago del Estero.
------------
In baghdad closest hotel is Babylon Rotana.
------------
In madimba closest hotel is Old Boma Hotel.
------------
In redlands closest hotel is Best Western Moreno Hotel & Suites.
------------
In kununurra

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map


#markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotel_info])
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))